In [1]:
import json
import re

import pandas as pd
from gensim.models.word2vec import Word2Vec

In [2]:
df = pd.read_csv("classification_articles.csv")

# Data preprocessing

In [3]:
df_classified = df.loc[~df["self_defined_category"].isna()]

In [4]:
df_classified.shape

(200, 7)

In [5]:
# https://stackoverflow.com/questions/17796446/convert-a-list-to-a-string-and-back
df_classified["sentences"] = df_classified["sentences"].apply(lambda s: eval(s))
df_classified["keyword_top10"] = df_classified["keyword_top10"].apply(lambda s: eval(s))
df_classified["locations"] = df_classified["locations"].apply(lambda s: eval(s))
df_classified["entity_address"] = df_classified["entity_address"].apply(lambda s: eval(s))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value

# Explore categories and keywords

In [6]:
df_classified["self_defined_category"].value_counts()

其它      79
自然景觀    51
人文藝術    38
娛樂購物    32
Name: self_defined_category, dtype: int64

In [7]:
other_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "其它", "sentences"]:
    other_sentences.extend(l)

In [8]:
series_other = pd.Series(other_sentences)
series_other.value_counts().head(10)

飯店    14
房間    12
空間    11
推薦    10
景點    10
住宿     9
美食     8
台中     8
早餐     7
設施     7
dtype: int64

In [9]:
series_other.value_counts()[series_other.value_counts() > 1]

飯店      14
房間      12
空間      11
推薦      10
景點      10
        ..
分鐘       2
quby     2
會館       2
汽車旅館     2
北投       2
Length: 99, dtype: int64

In [10]:
nature_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "自然景觀", "sentences"]:
    nature_sentences.extend(l)

In [11]:
series_nature = pd.Series(nature_sentences)
series_nature.value_counts().head(10)

景點        16
櫻花        10
2019       9
js         9
露營         8
步道         8
window     7
push       7
花況         6
門票         5
dtype: int64

In [12]:
series_nature.value_counts()[series_nature.value_counts() > 1]

景點       16
櫻花       10
2019      9
js        9
露營        8
         ..
2018      2
免費        2
泰安派出所     2
遊客        2
時間        2
Length: 64, dtype: int64

In [13]:
art_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "人文藝術", "sentences"]:
    art_sentences.extend(l)

In [14]:
series_art = pd.Series(art_sentences)
series_art.value_counts().head(10)

景點     11
園區      5
宜蘭      5
台灣      5
空間      4
小朋友     4
打卡      4
體驗      4
建築      4
ig      4
dtype: int64

In [15]:
series_art.value_counts()[series_art.value_counts() > 1]

景點        11
園區         5
宜蘭         5
台灣         5
空間         4
小朋友        4
打卡         4
體驗         4
建築         4
ig         4
博物館        3
台中         3
免費         3
武淵         3
水火同源       3
藝術         3
公園         3
閱讀         3
推薦         3
參觀         2
草皮         2
餐廳         2
音樂         2
鍾肇政        2
青鳥         2
華山         2
陀螺         2
diy        2
溜滑梯        2
台南         2
龍潭         2
門票         2
咖啡         2
火車         2
泡腳         2
展覽         2
宿舍         2
戲水         2
大碗公溜滑梯     2
書店         2
現代         2
日式建築       2
日式         2
桃園         2
沙雕         2
活動         2
創作         2
苗栗市        2
時間         2
協櫻         2
地景         2
機芯         2
幾米         2
指尖         2
客家         2
dtype: int64

In [16]:
shop_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "娛樂購物", "sentences"]:
    shop_sentences.extend(l)

In [17]:
series_shop = pd.Series(shop_sentences)
series_shop.value_counts().head(10)

小朋友    6
溜滑梯    5
公園     4
js     4
桃園     3
宜蘭     3
台中     3
中壢     3
特賣會    3
打卡     3
dtype: int64

In [18]:
series_shop.value_counts()[series_shop.value_counts() > 1]

小朋友       6
溜滑梯       5
公園        4
js        4
桃園        3
宜蘭        3
台中        3
中壢        3
特賣會       3
打卡        3
diy       3
小米        3
台南        3
美食        2
親子        2
參考        2
push      2
台灣        2
開幕        2
喜歡        2
吉米        2
遊戲場       2
大江購物中心    2
設施        2
id        2
球池        2
花蓮        2
可愛        2
window    2
門票        2
體驗        2
台中后里      2
觀光工廠      2
空間        2
dtype: int64

In [19]:
# Find intersections of top-10-common keywords accross categories
# (to be excluded in following predictions)
series_common = (
    series_other.value_counts().head(10).index
        .append(series_art.value_counts().head(10).index)
        .append(series_nature.value_counts().head(10).index)
        .append(series_shop.value_counts().head(10).index)
)
series_common.value_counts()

景點        3
小朋友       2
宜蘭        2
空間        2
js        2
台中        2
打卡        2
設施        1
門票        1
體驗        1
公園        1
美食        1
溜滑梯       1
台灣        1
推薦        1
房間        1
中壢        1
特賣會       1
push      1
住宿        1
花況        1
window    1
園區        1
早餐        1
ig        1
步道        1
櫻花        1
桃園        1
2019      1
建築        1
飯店        1
露營        1
dtype: int64

In [20]:
list_excluding_keywords = series_common.value_counts()[series_common.value_counts() > 1].index.tolist()
list_excluding_keywords

['景點', '小朋友', '宜蘭', '空間', 'js', '台中', '打卡']

In [21]:
# Keywords can't be locations either
list_excluding_locations = [s for l in df_classified["locations"] for s in l]
list_excluding_locations = list(set(list_excluding_locations))
list_excluding_locations

['陽明山天籟渡假酒店',
 '台灣穀堡',
 '集集火車站',
 '拉拉山恩愛農場',
 '白河',
 '湖口',
 '龍潭',
 '八卦山',
 '彰化',
 '新竹火車站',
 '東勢',
 '鰲峰山運動公園',
 '赤崁璽樓民宿',
 '大安區',
 '臺中市眷村文物館',
 '大甲溪',
 '花蓮',
 '瑞春醬油',
 '澎湖跨海大橋',
 '松山區',
 '鯉魚潭',
 '高雄',
 '箱根',
 '馬來西亞',
 '台北車站',
 '清水',
 '台灣',
 '小琉球',
 '美術館',
 '情人橋',
 '武荖坑風景區',
 '大湖',
 '梅花',
 '彩虹',
 '菱潭街興創基地',
 '大溪老街',
 '鹿場',
 '紅樹林站',
 '二崁聚落',
 '銅鑼',
 '信義區',
 '苗栗',
 '車路墘教會',
 '北屯',
 '小北家',
 '圓林仔社區',
 '捷運忠孝復興站',
 '草屯',
 '谷關',
 '新北',
 '綠園道',
 '竹北',
 '運動公園',
 '北投',
 '青埔',
 '西屯',
 '東大門夜市',
 '東海岸',
 '樹谷農場',
 '澎湖',
 '環保公園',
 '中山',
 '三重',
 '西區',
 '屏東',
 '蜜餞形象館',
 '巴黎',
 '中壢',
 '泰安',
 '綠川',
 '復興區',
 '新北市',
 '華山',
 '台中',
 '后里',
 '紅樓',
 '月眉糖廠',
 '玉井',
 '亞太飯店',
 '宜蘭',
 '南印度',
 '文化村',
 '東豐自行車綠廊',
 '日本',
 '南投',
 '客家文物館',
 '南庄',
 '美國',
 '大甲',
 '眺望',
 '世界',
 '台東',
 '秘密花園',
 '松山',
 '新社',
 '自由天地民宿',
 '曼谷',
 '新化',
 '南門',
 '觀光夜市',
 '三峽',
 '青草湖',
 '南寮',
 '雲林',
 '嘉義',
 '童話世界',
 '鳥人創意旅店',
 '桃園',
 '易家仙人掌冰',
 '台江國家公園遊客中心',
 '勤美術館',
 '大安',
 '準園休閒農場',
 '西螺',
 '新竹',
 '室內',
 '礁溪',
 '淡水',
 '陽明山',
 '月亮',
 '水舞行館',
 '糖廠',

In [22]:
df_classified["sentences"] = df_classified["sentences"].apply(
    lambda l: [
        s for s in l
        # Excluding common keywords
        if s not in list_excluding_keywords and
        # Excluding locations
        s not in list_excluding_locations and
        # Excluding pure digits
        not re.match(r"\d+$", s)
    ]
)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Word2Vec

In [23]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except KeyError:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [24]:
model = Word2Vec(df_classified["sentences"], size=250, iter=10, min_count=2, sg=1, window=1)

In [25]:
most_similar(model, ["櫻花", "溜滑梯", "門票", "飯店", "民宿", "步道", "芒果"])

,櫻花,cos,溜滑梯,cos,門票,cos,飯店,cos,民宿,cos,步道,cos,芒果,cos
0,老闆,0.171519,孩子們,0.190876,創作,0.177436,可愛,0.245765,開幕,0.169111,度空間,0.171941,夜市,0.160729
1,溜滑梯,0.166381,建築,0.178039,生態,0.161908,戶外,0.171931,溫泉,0.168815,老闆,0.157370,火鍋,0.131234
2,創作,0.148122,櫻花,0.166381,咖啡,0.147168,戲水,0.155212,營業時間,0.155166,規劃,0.157066,追蹤,0.120545
3,大碗公溜滑梯,0.146144,繡球花,0.155903,小孩,0.120985,花園,0.141483,體驗,0.141767,波斯菊,0.147965,花況,0.118640
4,天空步道,0.145654,不錯,0.155141,營地,0.120963,美學,0.122866,時間,0.140328,泡腳,0.137799,精油,0.117088
5,波屋,0.143052,動物,0.153677,視野,0.120572,適合,0.120857,博物館,0.133179,暑假,0.134227,時間,0.112505
6,日式,0.124640,咖波,0.147440,日式建築,0.115677,窯烤麵包,0.119883,觀光工廠,0.133160,景觀餐廳,0.129025,青鳥,0.109626
7,動物,0.121347,window,0.129823,夢幻,0.111356,裝備,0.116096,地景,0.130396,入住,0.123155,料理,0.109209
8,停車場,0.120610,波斯菊,0.129169,裝備,0.106022,pizza,0.111634,大碗公溜滑梯,0.118400,機芯,0.120031,pizza,0.108668
9,吃起來,0.117851,老闆,0.127834,吃起來,0.105889,用餐,0.105593,落羽松,0.114980,台中后里,0.118479,協櫻,0.106795


In [26]:
dict_df_classified = df_classified.where(df_classified.notnull(), None).to_dict()
with open("preprocessed_data.json", "w") as f:
    json.dump(dict_df_classified, f, indent=2, ensure_ascii=False)

In [27]:
model.save("word2vec.model")